In [1]:
from datetime import datetime

from libs.configuration import configure
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

from shared.spark_config import create_spark_config

env = configure()
conf = create_spark_config("M2_Processors.airport.tier1")

{'DATASTORAGE_AZURE_ACCESSKEY': 'dYj3PCGRyYHYR9pRo2xPa9BQ7t/sUonACJ29wcPdpa+1IC70rMbZmpfpHJJWJtk7OeASL+eQDIWq+AStGg3CCA==',
 'DATASTORAGE_AZURE_ACCOUNTNAME': 'icebergpe6t5cug7x9f45p8',
 'KAFKA_BOOTSTRAP_SERVERS': 'localhost:9092',
 'KAFKA_SCHEMA_REGISTRY_URL': 'http://localhost:8081',
 'KAFKA_SESSION_TIMEOUT_MS': '45000',
 'KAFKA_TOPIC_RAW_AIRCRAFT': 'raw.aircraft',
 'KAFKA_TOPIC_RAW_AIRLINE': 'raw.airline',
 'KAFKA_TOPIC_RAW_AIRPORT': 'raw.airport',
 'KAFKA_TOPIC_RAW_FLIGHT_SIGNAL': 'raw.flight_signal',
 'KAFKA_TOPIC_T1_AIRCRAFT': 't1.aircraft',
 'KAFKA_TOPIC_T1_AIRLINE': 't1.airline',
 'KAFKA_TOPIC_T1_AIRPORT': 't1.airport',
 'KAFKA_TOPIC_T1_FLIGHT_SIGNAL': 't1.flight_signal',
 'KAFKA_TOPIC_T2_AIRCRAFT': 't2.aircraft',
 'KAFKA_TOPIC_T2_AIRLINE': 't2.airline',
 'KAFKA_TOPIC_T2_AIRPORT': 't2.airport',
 'KAFKA_TOPIC_T2_FLIGHT_SIGNAL': 't2.flight_signal',
 'POSTGRE_JDBC_URL': 'jdbc:postgresql://localhost:5432/iceberg-catalog',
 'POSTGRE_PASSWORD': 'grebeci',
 'POSTGRE_USER': 'iceberg',
 

In [2]:
spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [3]:
df = (
    spark.readStream.format("iceberg").option("stream-from-timestamp", "1705017600")
    # .option("stream-from-timestamp", datetime.now().timestamp())
    .load("dev.raw.flight_signals")
)

In [ ]:
df.describe().show()

In [4]:
df = df.where(F.col("icao24").isNotNull() & F.col("lat").isNotNull() & F.col("lon").isNotNull())

In [5]:
df = df.withColumn("created_ts", F.current_timestamp()).withColumn(
    "updated_ts", F.current_timestamp()
)

In [6]:
write_stream = (
    df.writeStream.format("iceberg")
    .outputMode("append")
    .trigger(processingTime="10 seconds")
    .option(
        "checkpointLocation",
        f"abfss://warehouse@{env.DATASTORAGE_AZURE_ACCOUNTNAME}.dfs.core.windows.net/_checkpoints/{env.KAFKA_TOPIC_T1_FLIGHT_SIGNAL}",
    )
    .toTable("dev.tier1.flight_signals")
)

In [8]:
write_stream.stop()

False